### Лабораторная работа №2
## Тема: Каркасная визуализация выпуклого многогранника. Удаление невидимых линий.
Черных Сергей М8О-305Б-20

Вариант: Правильный октаэдр

Разработать формат представления многогранника и процедуру его каркасной отрисовки в ортографической и изометрической проекциях. Обеспечить удаление невидимых линий и возможность пространственных поворотов и масштабирования многогранника. Обеспечить автоматическое центрирование и изменение размеров изображения при изменении размеров окна.

In [31]:
import plotly
import numpy as np
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import plotly.graph_objs as go
import chart_studio.plotly as py


In [32]:
class Prism(object):
    def __init__(self,u,v,w):
        self.u = np.array(u)
        self.v = np.array(v)
        self.w = np.array(w)

    def goprism(self):
        x = [0,self.u[0], self.u[0]+self.v[0], self.v[0], self.w[0], self.w[0]+self.u[0], self.w[0]+self.u[0]+self.v[0],\
             self.v[0]+self.w[0]]
        y = [0, self.u[1], self.u[1]+self.v[1], self.v[1], self.w[1], self.w[1]+self.u[1], self.w[1]+self.u[1]+self.v[1],\
             self.v[1]+self.w[1]]
        z = [0, self.u[2],self.u[2]+self.v[2], self.v[2], self.w[2], self.w[2]+self.u[2], self.w[2]+self.u[2]+self.v[2],\
             self.v[2]+self.w[2]]
        prism = [go.Mesh3d(x=x,y=y,z=z,
            i = [0, 0, 3, 4, 4, 4, 4, 4, 5, 6, 6, 7],
            j = [2, 3, 4, 3, 6, 7, 1, 5, 2, 2, 7, 3],
            k = [1, 2, 0, 7, 5, 6, 0, 1, 1, 5, 2, 2],
            opacity = 1.0)
            ]
        return prism

    def lytprism(self, name_title):
        ubx = sum([abs(self.u[0]),abs(self.v[0]),abs(self.w[0])])
        uby = sum([abs(self.u[1]),abs(self.v[1]),abs(self.w[1])])
        ubz = sum([abs(self.u[2]),abs(self.v[2]),abs(self.w[2])])
        layout = go.Layout(
            title = name_title,
            scene = dict(xaxis=dict(
                title='x', range=[-ubx-5,ubx+5]
            ),
            yaxis=dict(
                title='y',range=[-uby-5,uby+5]
            ),
            zaxis = dict(
                title='z',range=[-ubz-5,ubz+5])
            )              
        )
        return layout
    
class Rectangle(object):
    def __init__(self, x0, x1, y0, y1):
        self.x0 = x0
        self.x1 = x1
        self.y0 = y0
        self.y1 = y1

    def gorect(self):
        rect = [go.Scatter(
            x=[1.5, 4.5],
            y=[0.75, 0.75],
        )]
        return rect

    def lytrec(self, name_title):
        layout = {
            'title': name_title,
            'xaxis': {
                'range': [-10.0, 10.0],
                'showgrid': False,
            },
            'yaxis': {
                'range': [-10.0, 10.0]
            },
            'shapes': [
                {
                    'type': 'rect',
                    'x0': self.x0,
                    'y0': self.y0,
                    'x1': self.x1,
                    'y1': self.y1,
                    'line': {
                        'color': 'rgba(63, 127, 191, 1)',
                        'width': 2,
                },
                    'fillcolor': 'rgba(63, 127, 191, 1)',
                },
            ]
        }
        return layout

In [33]:
my_prism = Prism([4.,0.,0.],[0.,4.,0.],[0.,0.,4.])
data = my_prism.goprism()
layout_3d = my_prism.lytprism("4-угольная прямая правильная призма")

prism_x = Rectangle(0, 4, 0, 8)
prism_y = Rectangle(0, 4, 0, 8)
prism_z = Rectangle(0, 4, 0, 4)
data_x = prism_x.gorect()
data_y = prism_y.gorect()
data_z = prism_z.gorect()
layout_x = prism_x.lytrec("Проекция на плоскость YOZ")
layout_y = prism_y.lytrec("Проекция на плоскость XOZ")
layout_z = prism_z.lytrec("Проекция на плоскость XOY")

fig = go.Figure(data=data, layout=layout_3d)
fig_x = {
    'data': data_x,
    'layout': layout_x,
}
fig_y = {
    'data': data_y,
    'layout': layout_y,
}
fig_z = {
    'data': data_z,
    'layout': layout_z,
}

In [34]:
iplot(fig, filename='prism')
iplot(fig_x, filename='rect_x')
iplot(fig_y, filename='rect_y')
iplot(fig_z, filename='rect_z')

In [85]:
import plotly.graph_objects as go
from math import sqrt


vertices = np.array([
        ( 1,  0,  0),
        ( 0,  1,  0),
        ( 0,  0,  1),
        (-1,  0,  0),
        (0, -1,  0),
        ( 0,  0, -1)
        ]
    )

faces = np.array([
        (0, 1, 2),
        (1, 3, 2),
        (3, 4, 2),
        (4, 0, 2),
        (0, 5, 1),
        (1, 5, 3),
        (3, 5, 4),
        (4, 5, 0)
        ]
    )
def plot_3d(a):
    fig = go.Figure(data=[
        go.Mesh3d(
            # 8 vertices of a cube
            x=a*vertices[:,0],
            y=a*vertices[:,1],
            z=a*vertices[:,2],
            colorbar_title='z',

            # The intensity is based on the distance
            intensity=np.linalg.norm(vertices, axis=1),
            colorscale=[(0.0, 'rgb(0, 0, 55)'),  (1.0, 'rgb(0, 0, 175)'),],
            # i, j and k give the vertices of triangles
            i=faces[:,0],
            j=faces[:,1],
            k=faces[:,2],
            name='y',
            showscale=False,
            flatshading=True,
        )
    ])

    fig.show()

plot_3d(1)